In [1]:
# Import required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torchvision import transforms
from PIL import Image
import os

# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Load Data from Google Drive
# Set paths to your files on Google Drive (adjust the file names as needed)
morskie_oko_mapping_path = '/content/drive/My Drive/MorskieOko_MappingLayers_Stats.csv'
summary_stats_path = '/content/drive/My Drive/MorskieOko_summary_stats.csv'
yearly_water_area_path = '/content/drive/My Drive/MorskieOko_Yearly_Water_Area.csv'
summary_stats_from_geotiffs_path = '/content/drive/My Drive/MorskieOko_Summary_Stats__from_GeoTIFFs_.csv'
yearly_water_area_clean_path = '/content/drive/My Drive/Yearly_Water_Area_clean.csv'

# Load the CSV files
morskie_oko_mapping = pd.read_csv(morskie_oko_mapping_path)
summary_stats = pd.read_csv(summary_stats_path)
yearly_water_area = pd.read_csv(yearly_water_area_path)
summary_stats_from_geotiffs = pd.read_csv(summary_stats_from_geotiffs_path)
yearly_water_area_clean = pd.read_csv(yearly_water_area_clean_path)

# 3. Merge Dataframes
data = pd.merge(morskie_oko_mapping, summary_stats, on='year', how='inner')
data = pd.merge(data, yearly_water_area, on='year', how='inner')

# Select relevant features for prediction
features = data[['occurrence_mean', 'change_abs_mean', 'water_area_mean']]
labels = data['pollution_level']  # Assume 'pollution_level' is the target

# 4. Data Preprocessing
# Handle missing values by filling with the mean
features = features.fillna(features.mean())
labels = labels.fillna(labels.mean())

# Normalize the features
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Convert to PyTorch tensors
features = torch.tensor(features, dtype=torch.float32)
labels = torch.tensor(labels.values, dtype=torch.float32).view(-1, 1)  # Reshape for regression

# 5. Image Preprocessing
# Define transformations for the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for pre-trained models
])

# 6. Dataset Class for PyTorch
class WaterPollutionDataset(Dataset):
    def __init__(self, features, labels, image_dir, transform=None):
        self.features = features
        self.labels = labels
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        # Get tabular features
        tabular_data = self.features[idx]
        label = self.labels[idx]

        # Get corresponding image (assuming filenames are indexed by 'idx')
        image_path = os.path.join(self.image_dir, f"{idx + 1}.png")  # Images are named 1.png, 2.png, etc.
        image = Image.open(image_path)

        # Apply transformations to image
        if self.transform:
            image = self.transform(image)

        return tabular_data, image, label

# Image directory (adjust as needed)
image_dir = '/content/drive/My Drive/'  # Make sure the images are in this directory

# 7. Split Data into Training and Test Sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create datasets and dataloaders
train_dataset = WaterPollutionDataset(train_features, train_labels, image_dir, transform)
test_dataset = WaterPollutionDataset(test_features, test_labels, image_dir, transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 8. Define the Model
class WaterPollutionNN(nn.Module):
    def __init__(self):
        super(WaterPollutionNN, self).__init__()

        # Image processing (CNN part)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected part for tabular data
        self.fc_tabular = nn.Linear(3, 32)  # 3 input features

        # Final fully connected layer
        self.fc1 = nn.Linear(64*56*56 + 32, 128)  # 64*56*56 is the output of the CNN, plus the tabular features
        self.fc2 = nn.Linear(128, 1)  # Output one value (pollution level)

    def forward(self, x_tabular, x_image):
        # Image processing
        x_image = self.pool(torch.relu(self.conv1(x_image)))
        x_image = self.pool(torch.relu(self.conv2(x_image)))
        x_image = x_image.view(x_image.size(0), -1)  # Flatten

        # Tabular processing
        x_tabular = torch.relu(self.fc_tabular(x_tabular))

        # Concatenate image and tabular features
        x = torch.cat((x_image, x_tabular), dim=1)

        # Fully connected layers
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = WaterPollutionNN()

# 9. Training Setup
# Define loss function and optimizer
criterion = nn.MSELoss()  # For regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 10. Training the Model
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs_tabular, inputs_image, targets in train_loader:
        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs_tabular, inputs_image)

        # Calculate loss
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

# 11. Testing the Model
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for inputs_tabular, inputs_image, targets in test_loader:
        outputs = model(inputs_tabular, inputs_image)
        predictions.append(outputs)
        true_labels.append(targets)

# Flatten the predictions and true labels for comparison
predictions = torch.cat(predictions).numpy()
true_labels = torch.cat(true_labels).numpy()

# You can print or visualize the predictions and true labels
print("Predictions: ", predictions)
print("True Labels: ", true_labels)

# 12. Save the Model
torch.save(model.state_dict(), '/content/drive/My Drive/water_pollution_model_with_images.pth')


ValueError: mount failed